# Select and visualize an item from the Nitrogen Dioxide (NO_2) collection

In [1]:
import requests as re
import folium

STAC_ENDPOINT_URL = "https://staging-stac.delta-backend.com"
RASTER_ENDPOINT_URL = "https://staging-raster.delta-backend.com"

In [2]:
no2_collection_summary = re.get(f"{STAC_ENDPOINT_URL}/collections/no2-monthly").json()
no2_collection_summary

{'id': 'no2-monthly',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'}],
 'title': 'NO₂',
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01T00:00:00Z',
     '2022-01-01T00:00:00Z']]}},
 'license': 'MIT',
 'summaries': {'datetime': ['2016-01-01T00:00:00Z', '2022-05-01T00:00:00Z'],
  'cog_default': {'max': 50064805976866816, 'min': -6618294421291008}},
 'description': 'Darker colors indicate higher nitrogen dioxide (NO₂) levels and more activity. Lighter colors indicate lower levels of NO₂ an

In [3]:
rescale_values = no2_collection_summary["summaries"]["cog_default"]
rescale_values

{'max': 50064805976866816, 'min': -6618294421291008}

In [4]:
response = re.post(
    f"{STAC_ENDPOINT_URL}/search", 
    json={
        "collections": ["no2-monthly"], 
        "query": {"datetime": {"eq":"2021-01-01T00:00:00"}},
        "limit": 100
    }
).json()
item = response["features"][0]
item

{'id': 'OMI_trno2_monthly_0.10x0.10_202101_Col3_V4.nc',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly/items/OMI_trno2_monthly_0.10x0.10_202101_Col3_V4.nc'}],
 'assets': {'cog_default': {'href': 's3://veda-data-store-staging/no2-monthly/OMI_trno2_monthly_0.10x0.10_202101_Col3_V4.nc.tif',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
   'roles': ['data', 'layer'],
   'raster:bands': [{'scale': 1.0,
     'nodata': -1.2676506002282294e+30,
     'offset': 0.0,

In [5]:
tiles = re.get(
    f"{RASTER_ENDPOINT_URL}/stac/tilejson.json?collection={item['collection']}&item={item['id']}&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=rdbu_r&rescale=0%2C15000000000000000", 
).json()
tiles

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://staging-raster.delta-backend.com/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=no2-monthly&item=OMI_trno2_monthly_0.10x0.10_202101_Col3_V4.nc&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=rdbu_r&rescale=0%2C15000000000000000'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

In [6]:
m = folium.Map(
    zoom_start=2,
    scroll_wheel_zoom=True, 
    tiles=tiles["tiles"][0], 
    attr="VEDA", 
    minzoom=18, 
    maxzoom=0, 
    
)

m